<a href="https://colab.research.google.com/github/swarris/simulitis/blob/master/Simulitis_EN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spread of Simulitis
In this project we will simulate the spread of an imaginairy virus. 
This virus, *simulitis* can be transmitted for person to person and can be fatal. The transmission rate and fatality rate can be changed per simulation. These simulations are written in the programming language Python. These simulations are **illustrative** are intented to **teach programming in Python**. The used models are **not scientific**! 

Idea based on the [simulation of the Washington Post](https://www.washingtonpost.com/graphics/2020/world/corona-simulator/)

## Teaching goals
The simulations are created in Python and make use of the iPython Notebook technology. These notebooks run in a webbrowser and are very useful for experimenting with Python. You can step through your code, add new code or remove parts and test the results. There are many examples on how to use notebooks and Python, for example [Learn Python 3 with Jupyter Notebook ](https://gist.github.com/kenjyco/69eeb503125035f21a9d).
The simulations are writting using **object oriented programming**, a way of structuring your code. This notebook also allows you to experiment with **making animations and graphs**. 

## Exercises
This notebook contains several exercises. It is possible to run the simulations without doing the programming exercises! I would recommend going through all the simulations before starting at the programming exercises.

## Let's create a world
Out worlds is rectangular and *X* by *Y* wide. The infection distance give the minimal distance a person much have to an infected person to become infected himself. Infection is absolute: when you get within this range you are infected.

In [0]:
class World:

  def __init__(self, X, Y, numberOfPersons, infectionDistance):
    """ 
    constuctor with the size of the world, the number of people in it and
    the infection distance in which people get infected.
    """
    self.X = X
    self.Y = Y
    self.numberOfPersons = numberOfPersons
    self.infectionDistance = infectionDistance
    # we start with an empty population:
    self.population = []

  def create(self):
    """ 
    This method create a population with #numberOfPersons and one sick person.
    """
    self.population = []
    for i in range(self.numberOfPersons):
      self.population.append(Person(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    # make one sick
    self.population[0].infected = True


### Exercises


1.   What is the maximum age a person can get?
2.   Where are the persons placed? Do they start in the same spot or not?
3.   **Programming**: make a random number of people sick. Use `random.random()` 
4.   **Programming**: divide all persons across three different circles. 




## Now let's define a Person 
A person has an age and a startLocation in the world. A person can also walk about.

In [0]:
import math
import random

class Person:
  def __init__(self, age, world, startX, startY):  
    """
    A Person has an age, a world to live in and 
    a start position in this world. He/she moves in a random direction. 
    A Person is not infected when it is created.
    """
    self.age = age
    self.world = world
    self.X = startX
    self.Y = startY
    # pick a random walkingDirection:
    self.walkingDirection = random.random()*math.pi*2.0
    # A person is created in good health:
    self.infected = False

    
  def takeStep(self):
    """
    This method makes sure this person takes a step in the choosen direction.
    In some cases it changes direction.
    """

    # Change of direction?
    if random.random() > 0.95:
      self.walkingDirection = random.random()*math.pi*2.0
    # Take a step
    self.X = self.X + math.cos(self.walkingDirection)
    self.Y = self.Y + math.sin(self.walkingDirection)
    # make sure the person does not walk off the world:
    if self.X > self.world.X:
      self.X = 0
    elif self.X < 0:
      self.X = self.world.X
    if self.Y > self.world.Y:
      self.Y = 0
    elif self.Y < 0:
      self.Y = self.world.Y

        
  def currentLocation(self):
    """ Where am I now? """
    return [self.X, self.Y]

  def infect(self):
    """ 
    Check all people in the world: if I'm close enough and I'm infected
    make that person sick. Returns the number of infected persons.
    """
    infectedPersons = 0
    if self.infected:
      for p in self.world.population:
        # if the person is within the infection distance and not yet infected, infect this person:
        if not p.infected and math.sqrt((p.X-self.X)**2 + (p.Y-self.Y)**2) <= self.world.infectionDistance :
          p.infected = True
          infectedPersons += 1
    return infectedPersons


### Exercises

1.  Create a drawing on paper of a world with several people. Indicate with arrows where they will go once they tend to leave the world. 
2.   **Programming**: Imagine the world is surrounded by a wall. Let a person bounce off the wall in stead of making him/her appear on the other side.
3.   **Programming**: add a 50/50 chance of getting infected once a person gets with infectionDistance.



## The people on the world
Now we can create a world and a person we can combine the two. With the following variables you can indicate the size of the world and how many person are in the world. Tip: the bigger the *world* and/or the more people are in it, the longer it will take for the simulation to end.

In [0]:
worldSize = 100
numberOfPersons = 100
infectionDistance = 3.0
timeStepsInTheAnimation = 500

### Exercise

1.   Play with the values for `worldSize`, `numberOfPersons` and `infectionDistance`. Check for instance how the affect running time of the simulation below. 



Now we can create a world with one infected person:

In [0]:
world = World(worldSize,worldSize, numberOfPersons, infectionDistance)


### Exercises

1.   The current world is square. How can you tell?
2.   **Programming**: add another variable to be able to create a rectangular world.



The next piece of code is necessary to create the animation. The person walk through the world and start infecting others.

In [0]:
%%capture
# First we need some additional libraries:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML

class Animation:
  def __init__(self, world, timeStepsInTheAnimation):
    """
    Definitie van het plot frame.
    """
    self.world = world
    self.timeStepsInTheAnimation = timeStepsInTheAnimation
    # Define the figure:
    self.fig, (self.ax, self.ax2) = plt.subplots(2,1,figsize=(8,12))
    # Set limits:
    self.ax.set_xlim(( 0, world.X))
    self.ax.set_ylim((0, world.Y))
    self.ax2.set_xlim(( 0, timeStepsInTheAnimation))
    self.ax2.set_ylim((0, 100))

    # We keep track of the number of healthy and sick persons:
    self.healthy, = self.ax.plot([], [], "bo", lw=2)
    self.sick, = self.ax.plot([], [], "ro", lw=2)
    self.totalSick, = self.ax2.plot([], [], "ro", lw=2)
    # The legend is positioned between the two figures
    self.legend = self.ax2.legend((self.healthy, self.sick), 
                                  ("Healthy", "Sick"), 
                                   loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=2)
    # We also keep track of the number of sick person per time point:
    self.totalSickStatus = []

  def init(self):
    """ Make data empty for the figure """
    self.healthy.set_data([], [])
    self.sick.set_data([], [])
    self.totalSick.set_data([],[])
    return (self.healthy,self.sick, self.totalSick)

  def animate(self, i):
    """ 
    Creates the animation and lets people walk around and infect others.
    """
    healthyX = []
    healthyY = []
    sickX = []
    sickY = []
    # We start with zero sick persons:
    self.totalSickStatus.append(0.0)

    # Walk around
    for p in self.world.population:
      p.takeStep()
    # Infect others
    for p in self.world.population:
      p.infect()
    
    # Check current situation
    for p in self.world.population:
      location = p.currentLocation()
      # Is this person infected?
      if p.infected:
        # Add the location of this sick person to the list:        
        sickX.append(location[0])
        sickY.append(location[1])
        # And increase the number of sick people:
        self.totalSickStatus[-1] += 1.0
      else:
        # Add the location of this healthy person to the list:
        healthyX.append(location[0])
        healthyY.append(location[1])
    
    self.healthy.set_data(healthyX, healthyY)
    self.sick.set_data(sickX, sickY)
    # Scale the data by the size of the population :
    self.totalSickStatus[-1] = self.totalSickStatus[-1] /len(world.population) * 100.0
    self.totalSick.set_data(list(range(0,i+1)), self.totalSickStatus)
    return (self.healthy, self.sick, self.totalSick)

  def createFrame(self):
    self.frame = animation.FuncAnimation(self.fig, self.animate, init_func=self.init,
                               frames=self.timeStepsInTheAnimation, interval=100, blit=True)
 

## The animation
It might take a while before the animation has been create, so please be patient. Change *to_jshtml* to *to_html5_video* to create a mpeg video. 

In [0]:
%%capture
worldAnimation = Animation(world, timeStepsInTheAnimation)

In [0]:
world.create()
worldAnimation.createFrame()
HTML(worldAnimation.frame.to_jshtml())

### Exercises


1.   Will everybody get infected with the current settings? Change the  settings, for example, `numberOfPersons`, and see if you can prevent everybody  from getting infected in the end. 
2.   What is the shape of the graph usually? Is it mainly linear? Exponential? Or does it have the shape of an S-curve?
3.   **Programming**: Draw the `infectionDistance` around every sick person.
4.   **Programming**: _Social distancing_: make sure everybody stays out of each others way. You can do this for instance to make this person to take a step back when he/she is within twice the `infectionDistance`.
5.   **Programming**: Give everybody  who is doing this _social distancing_ temporarily a different color. 



## Immunity
People become cured after some time and therefore become immune for simulitis. This behavior is set in the method `becomesBetter`. We now also need to implement a `BetterWorld` and also the animation needs to adjusted.

In [0]:
%%capture
class WithImmuneSystem(Person):
  def __init__(self, age, world, startX, startY):
    super().__init__(age, world, startX, startY)
    self.immune = False
    self.stepsTaken = 0
  
  def becomesBetter(self, numberOfSteps):
    """ A person becomes better and therefore immune after some time. """
    if self.stepsTaken >= numberOfSteps and self.infected:
      self.infected = False
      self.immune = True

  def infect(self):
    """ 
    Check all people in the world: if I'm close enough and I'm infected
    make that person sick. Returns the number of infected persons.
    """
    infectedPersons= 0
    if self.infected:
      for p in self.world.population:
        if not p.infected and not p.immune and math.sqrt((p.X-self.X)**2 + (p.Y-self.Y)**2) <= self.world.infectionDistance :
          p.infected = True
          infectedPersons += 1
    return infectedPersons

  def takeStep(self):
    """ Now also keep track of the number of steps taken. """
    super().takeStep()
    if self.infected:
      self.stepsTaken += 1    


class BetterWorld(World):
  """ A BetterWorld consists of people who can get better and immune """
  def __init__(self, X, Y, numberOfPersons, infectionDistance):
    super().__init__(X, Y, numberOfPersons, infectionDistance)

  def create(self):
    self.population = []
    for i in range(self.numberOfPersons):
      self.population.append(WithImmuneSystem(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    self.population[0].infected = True

class BetterAnimation(Animation):
  def __init__(self, world, timeStepsInTheAnimation, numberOfSteps):
    super().__init__(world, timeStepsInTheAnimation)
    self.immune, = self.ax.plot([], [], "go", lw=2)
    self.totalImmune, = self.ax2.plot([], [], "go", lw=2)
    self.legend.remove()
    self.legend = self.ax2.legend((self.healthy, self.sick, self.immune), 
                                  ("Healthy", "Sick", "Better & immune"),
                                  loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=3)
    self.totalImmuneStatus = []
    self.numberOfSteps = numberOfSteps
    

  def init(self):
    super().init()
    self.immune.set_data([], [])
    self.totalImmune.set_data([], [])
    return (self.healthy,self.sick, self.immune, self.totalImmune)

  def animate(self, i):
    healthyX = []
    healthyY = []
    sickX = []
    sickY = []
    immuneX = []
    immuneY = []
    self.totalSickStatus.append(0.0)
    self.totalImmuneStatus.append(0.0)
    
    for p in self.world.population:
      p.takeStep()
    for p in self.world.population:
      p.infect()
    for p in self.world.population:
      p.becomesBetter(self.numberOfSteps)

    
    for p in self.world.population:
      location = p.currentLocation()
      if p.infected:        
        sickX.append(location[0])
        sickY.append(location[1])
        self.totalSickStatus[-1] += 1.0
      elif p.immune:
        immuneX.append(location[0])
        immuneY.append(location[1])
        self.totalImmuneStatus[-1] += 1.0
      else:
        healthyX.append(location[0])
        healthyY.append(location[1])

    self.healthy.set_data(healthyX, healthyY)
    self.sick.set_data(sickX, sickY)
    self.totalSickStatus[-1] = self.totalSickStatus[-1] /len(world.population) * 100.0
    self.totalSick.set_data(list(range(0,i+1)), self.totalSickStatus)    
    self.totalImmuneStatus[-1] = self.totalImmuneStatus[-1] /len(world.population) * 100.0
    self.totalImmune.set_data(list(range(0,i+1)), self.totalImmuneStatus)
    self.immune.set_data(immuneX, immuneY)

    return (self.healthy, self.sick, self.immune, self.totalSick, self.totalImmune)



Now we can create this `BetterWorld` and animate it. The variable `numberOfSteps` gives the number of steps a person has to take before getting better, and hence immune.

In [0]:
%%capture
numberOfSteps = 150
BetterWorld = BetterWorld(worldSize,worldSize, numberOfPersons, infectionDistance)
BetterAnimation = BetterAnimation(BetterWorld, timeStepsInTheAnimation, numberOfSteps) 


In [0]:
BetterWorld.create()
BetterAnimation.createFrame()
HTML(BetterAnimation.frame.to_jshtml())

### Exercises


1.   Change the value for `numberOfSteps` before somebody gets better and see how it impacts the spread of simulitis. When are people getting better and immune before the virus can spread and when is everybody getting sick?
2.   **Programming**: Change the simulation in such a way that a person only has a change of getting better after a `numberOfSteps`.
3.   **Programming**: Change the simulation in such a way that a person has a change of getting better at each step but becomes better for sure after  `numberOfSteps`.



## Death
In the next simulation people might also die from the disease. With every step the chance of a sick person dying increases with age: the older a person is the higher the chance of dying. To make this happen we need to create new type of person, a new type of world and a different annimation.

In [0]:
class Human(WithImmuneSystem):
  def __init__(self, age, world, startX, startY):
    super().__init__(age, world, startX, startY)
    self.alive = True

  def takeStep(self):
    """ Now also keep track of the number of steps taken. And possibly die 
    in the process """
    if self.alive:
      super().takeStep()
      if self.infected and self.stepsTaken % 50 == 0:
        self.alive = (random.random() * 100.0) > self.age
    if not self.alive:
      self.infected = False

class RealWorld(BetterWorld):
  """ The RealWord consists of people who can die. """
  def __init__(self, X, Y, numberOfPersons, infectionDistance):
    super().__init__(X, Y, numberOfPersons, infectionDistance)

  def create(self):
    self.population = []
    for i in range(self.numberOfPersons):
      self.population.append(Human(100*random.random(), self, self.X * random.random(),self.Y*random.random()))
    self.population[0].infected = True

class RealAnimation(BetterAnimation):
  def __init__(self, world, timeStepsInTheAnimation, numberOfSteps):
    super().__init__(world, timeStepsInTheAnimation, numberOfSteps)
    self.dead, = self.ax.plot([], [], "k+", lw=2)
    self.totalDeadStatus = []
    self.averageAgeStatus = []
    self.totalDead, = self.ax2.plot([], [], "k+", lw=2)
    self.averageAge, = self.ax2.plot([], [], "b-", lw=2)
    self.legend.remove()
    self.legend = self.ax2.legend((self.healthy, self.sick, self.immune, self.totalDead, self.averageAge), 
                                  ("Healthy", "Sick", "Better & immune", "Dead", "Average age"), 
                                  loc="lower left",bbox_to_anchor= (0.0, 1.01), borderaxespad=0., ncol=5)


  def init(self):
    super().init()
    self.dead.set_data([], [])
    self.totalDead.set_data([],[])
    self.averageAge.set_data([],[])
    return (self.healthy,self.sick, self.immune, self.dead, self.totalSick, self.totalImmune, self.totalDead, self.averageAge)

  def animate(self, i):
    healthyX = []
    healthyY = []
    sickX = []
    sickY = []
    immuneX = []
    immuneY = []
    deadX = []
    deadY = []

    self.totalSickStatus.append(0.0)
    self.totalImmuneStatus.append(0.0)
    self.totalDeadStatus.append(0.0)
    self.averageAgeStatus.append(0.0)

    
    for p in self.world.population:
      p.takeStep()
    for p in self.world.population:
      p.infect()
    for p in self.world.population:
      p.becomesBetter(self.numberOfSteps)

    
    for p in self.world.population:
      location = p.currentLocation()
      if not p.alive:
        deadX.append(location[0])
        deadY.append(location[1])
        self.totalDeadStatus[-1] += 1.0
      elif p.infected:        
        sickX.append(location[0])
        sickY.append(location[1])
        self.totalSickStatus[-1] += 1.0
      elif p.immune:
        immuneX.append(location[0])
        immuneY.append(location[1])
        self.totalImmuneStatus[-1] += 1.0
      else:
        healthyX.append(location[0])
        healthyY.append(location[1])
      if p.alive:
        self.averageAgeStatus[-1] += p.age

    self.healthy.set_data(healthyX, healthyY)
    self.sick.set_data(sickX, sickY)
    self.immune.set_data(immuneX, immuneY)
    self.dead.set_data(deadX, deadY)

    self.totalSickStatus[-1] = self.totalSickStatus[-1] /len(world.population) * 100.0
    self.totalSick.set_data(list(range(0,i+1)), self.totalSickStatus)    
    self.totalImmuneStatus[-1] = self.totalImmuneStatus[-1] /len(world.population) * 100.0
    self.totalImmune.set_data(list(range(0,i+1)), self.totalImmuneStatus)

    self.averageAgeStatus[-1] = self.averageAgeStatus[-1] /(len(world.population)-self.totalDeadStatus[-1])
    self.averageAge.set_data(list(range(0,i+1)), self.averageAgeStatus)
    self.totalDeadStatus[-1] = self.totalDeadStatus[-1] /len(world.population) * 100.0
    self.totalDead.set_data(list(range(0,i+1)), self.totalDeadStatus)    

    return (self.healthy, self.sick, self.immune, self.totalSick, self.totalImmune, self.dead, self.totalDead, self.averageAge)

Below this `RealWorld` is created and filled with `Human`s. After that you can start the animation.

In [0]:
%%capture
RealWorld = RealWorld(worldSize,worldSize, numberOfPersons, infectionDistance)
realAnimation = RealAnimation(RealWorld, timeStepsInTheAnimation, numberOfSteps)

In [0]:
RealWorld.create()
realAnimation.createFrame()
HTML(realAnimation.frame.to_jshtml())

### Exercises

1.   According to this code, when does a person die? 
2.   The basic settings in this notebook a quite harsh for the populations. Which percentage of people has died at the end of the simulation?
3.   Enlarge the the world somewhat without adding more people to it. How this affect somebody's chance of survival?
4.   **Programming**: make the chances of dying variable. 
5.   **Programming**: change the simulation in such a way that young people have a higher chance of dying. You should be able to see that in the line for the average age.
6.   **Programming**: make sick people walk slowly. How does this affect the spread? 
7.   **Programming**: Add _social distancing_ (see previous exercise) and see how it affects the spread of the virus.

